## MNIST Handwritten Digit Recognition using Astra and Pytorch

In [1]:
#imports
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import numpy as np

In [2]:
#Astra Pytorch Dataset Definition
class AstraDataset(Dataset):
    def __init__(self,
                cloud_config={},
                auth_provider=None,
                keyspace="",
                table="raw_train",
                length=0,
                transform=None):
        self.db = Cluster(cloud=cloud_config, auth_provider=auth_provider).connect()
        self.keyspace = keyspace
        self.table = table
        self.length = length
        self.transform = transform

    def __getitem__(self, index):
        x = np.float32(np.array([float(pixel) for pixel in self.db.execute("SELECT pixels from "+self.keyspace+"."+self.table+" WHERE id = "+str(index)+";").one()[0]]).reshape(28,28)/255)
        y = self.db.execute("SELECT label from "+self.keyspace+"."+self.table+" WHERE id = "+str(index)+";").one()[0]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return self.length



In [3]:
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle as pkl
import auth

In [4]:
cloud_config = {'secure_connect_bundle': auth.scb_path}
auth_provider = PlainTextAuthProvider(auth.auth_id, auth.auth_token)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

In [5]:
train_dataset = AstraDataset(   
                    cloud_config, 
                    auth_provider, 
                    "mnist_digits", 
                    "raw_train", 
                    100, 
                    transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))])
                             )
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

test_dataset = AstraDataset(
                    cloud_config, 
                    auth_provider, 
                    "mnist_digits", 
                    "raw_test", 
                    100, 
                    transforms.Compose([
                               transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))])
                             )
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)

In [6]:
n_epochs = 1
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

print(example_data.shape)

TypeError: 'NoneType' object is not subscriptable

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [ ]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [ ]:
def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            description_string = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item())
            train_losses.append(loss.item())
            train_counter.append(
                (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
            network_state = bytearray(pkl.dumps(network.state_dict()))
            optimizer_state = bytearray(pkl.dumps(optimizer.state_dict()))
            print(type(network_state))
            #torch.save(network.state_dict(), 'results/model.pth')
            #torch.save(optimizer.state_dict(), 'results/optimizer.pth')
            query = "INSERT INTO mnist_digits.models (id, network, optimizer, upload_date, comments) VALUES (uuid(), %s, %s, toTimestamp(now()), %s);"
            values = [network_state, optimizer_state, description_string]
            session.execute(query, values)

In [ ]:
def test():
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()